# Performance

This notebook covers topics of performance and foundation of what to think about when you want to juice out extra bit performance.

In this notebook we focus on CPU bound tasks specifically.

Examples given are in C#, but the general ideas should be applicable to most languages.

## Benchmark.NET

To optimize the performance we first need to be able to measure it. For this purpose we will be using the library [BenchmarkDotNet](https://github.com/dotnet/BenchmarkDotNet).

Quick link how to setup benchmarks: [getting started](https://benchmarkdotnet.org/articles/guides/getting-started.html).

## Memory allocations

- One of the easiest performance gains are just fixing the application level logic
- If the type explains that it is expected to be used in a specific way, then it is best to follow these instructions

```csharp
public class MemoryAllocations1
{
    [Benchmark]
    public List<int> AllocateIncrementally()
    {
        var list = new List<int>();

        for (int i = 0; i < 1_500_000; i++)
            list.Add(i);

        return list;
    }

    [Benchmark]
    public List<int> AllocateWithCapacity()
    {
        var list = new List<int>(1_500_000);

        for (int i = 0; i < 1_500_000; i++)
            list.Add(i);

        return list;
    }
}
```

```csharp
[MemoryDiagnoser]
public class MemoryAllocations2
{
    [Benchmark]
    public string StringWithConcatenation()
    {
        string result = "";
        for (int i = 0; i < 1000; i++)
        {
            result = string.Concat(result, i.ToString());
        }
        return result;
    }

    [Benchmark]
    public string StringWithStringBuilder()
    {
        var sb = new StringBuilder();
        for (int i = 0; i < 1000; i++)
        {
            sb.Append(i.ToString());
        }
        return sb.ToString();
    }
}
```

## Memory locality

When a value is read from memory, technically not only the exact value requested is read, but a range of values from memory which includes the requested value. The range is stored in CPU cache.

- The size of mentioned range is called *cache line*.
- Most common size of cache line is now *64 bytes*.

For example:
- You are trying to read byte value at an offset *9000* (dec) from memory
- Cache line including bytes *9000 - 9063* will be actually read.
- If you want to read byte at an offset *9001*, then it is already in the cache.

![CPU cache access speeds](assets/cpu_cache.gif)

Source: https://planetscale.com/blog/caching

Although it is hard to account for everything with small examples, this one tries to sum the numbers from a collection when it is allocated in the stack and when it is allocated on the heap.

```csharp
public class MemoryLocality1
{
    [Benchmark]
    public void StackBased()
    {
        Span<int> buffer = stackalloc int[100_000];
        for (int i = 0; i < buffer.Length; i++)
            buffer[i] = i;

        int sum = 0;
        for (int i = 0; i < buffer.Length; i++)
            sum += buffer[i];
    }

    [Benchmark]
    public void HeapBasedWithBoxing()
    {
        int[] buffer = new int[100_000];
        for (int i = 0; i < buffer.Length; i++)
            buffer[i] = i;

        object sum = 0;
        for (int i = 0; i < buffer.Length; i++)
            sum = (int)sum + buffer[i];
    }

    [Benchmark]
    public void HeapBasedNoBoxing()
    {
        int[] buffer = new int[100_000];
        for (int i = 0; i < buffer.Length; i++)
            buffer[i] = i;

        int sum = 0;
        for (int i = 0; i < buffer.Length; i++)
            sum += buffer[i];
    }
}
```

In this example, assume that array will always be allocated on the heap, but in the case of `struct`, the contained values should be placed adjacent in the heap, while that is not guaranteed with `class`.

```csharp
public class MemoryLocality2
{
    public struct NumberContainerStruct
    {
        public int Number;
    }

    public class NumberContainerClass
    {
        public int Number;
    }

    [Benchmark]
    public void StructArray()
    {
        NumberContainerStruct[] buffer = new NumberContainerStruct[100_000];
        for (int i = 0; i < buffer.Length; i++)
            buffer[i].Number = i;

        int sum = 0;
        for (int i = 0; i < buffer.Length; i++)
            sum += buffer[i].Number;
    }

    [Benchmark]
    public void ClassArray()
    {
        NumberContainerClass[] buffer = new NumberContainerClass[100_000];
        for (int i = 0; i < buffer.Length; i++)
            buffer[i] = new NumberContainerClass { Number = i };

        int sum = 0;
        for (int i = 0; i < buffer.Length; i++)
            sum += buffer[i].Number;
    }
}
```

```csharp
public class MemoryLocality3
{
    private readonly int[] IntegersOnHeap = Enumerable.Range(0, 100_000).ToArray();

    [Benchmark]
    public void HeapSpanBased()
    {
        Span<int> buffer = IntegersOnHeap;

        int sum = 0;
        for (int i = 0; i < buffer.Length; i++)
            sum += buffer[i];
    }

    [Benchmark]
    public void HeapArrayBased()
    {
        int[] buffer = IntegersOnHeap;

        int sum = 0;
        for (int i = 0; i < buffer.Length; i++)
            sum += buffer[i];
    }

    [Benchmark]
    public void HeapMemoryBased()
    {
        Memory<int> buffer = IntegersOnHeap;

        int sum = 0;
        for (int i = 0; i < buffer.Length; i++)
            sum += buffer.Span[i];
    }
}
```

```csharp
public class MemoryLocality3_2
{
    [Benchmark]
    public int[] HeapSpanBased()
    {
        int[] buffer = new int[1_000_000];

        Span<int> span = buffer;
        for (int i = 0; i < span.Length; i++)
            span[i] = i;

        return buffer;
    }

    [Benchmark]
    public int[] HeapArrayBased()
    {
        int[] buffer = new int[1_000_000];

        for (int i = 0; i < buffer.Length; i++)
            buffer[i] = i;

        return buffer;
    }

    [Benchmark]
    public Memory<int> HeapMemoryBased()
    {
        Memory<int> buffer = new int[1_000_000];

        Span<int> span = buffer.Span;
        for (int i = 0; i < span.Length; i++)
            span[i] = i;

        return buffer;
    }
}
```

```csharp
public class MemoryLocality4
{
    [Benchmark]
    public void InterleavedArrayTuple()
    {
        var buffer = new (int A, int B)[1_000_000];
        for (int i = 0; i < buffer.Length; i++)
        {
            buffer[i].A = i;
            buffer[i].B = i * 2;
        }

        int sumA = 0;
        int sumB = 0;
        for (int i = 0; i < buffer.Length; i++)
        {
            sumA += buffer[i].A;
            sumB += buffer[i].B;
        }
    }

    public struct NumberPairStruct
    {
        public int A;
        public int B;
    }

    [Benchmark]
    public void InterleavedArrayStruct()
    {
        var buffer = new NumberPairStruct[1_000_000];
        for (int i = 0; i < buffer.Length; i++)
        {
            buffer[i].A = i;
            buffer[i].B = i * 2;
        }

        int sumA = 0;
        int sumB = 0;
        for (int i = 0; i < buffer.Length; i++)
        {
            sumA += buffer[i].A;
            sumB += buffer[i].B;
        }
    }

    [Benchmark]
    public void SplitArray()
    {
        var bufferA = new int[1_000_000];
        var bufferB = new int[1_000_000];
        for (int i = 0; i < bufferA.Length; i++)
        {
            bufferA[i] = i;
            bufferB[i] = i * 2;
        }

        int sumA = 0;
        int sumB = 0;
        for (int i = 0; i < bufferA.Length; i++)
        {
            sumA += bufferA[i];
            sumB += bufferB[i];
        }
    }
}
```

## Instruction pipelining and branch prediction

- Each stage of pipeline is executed by a different part of the CPU core
- For instruction to be complete it needs to move through all the stages of the pipeline
- If only 1 instruction is executed at the time in pipelined CPU, then it means that some parts of CPU are idling
- By starting next instruction before waiting the previous one to complete, CPU utilization could be increased
- Next instruction is not always obvious (think `JMP IF`)



| Cycle | Fetch | Decode | Execute | Write Back |
|-------|-------|--------|---------|------------|
| 1 | I1 | - | - | - |
| 2 | I2 | I1 | - | - |
| 3 | I3 | I2 | I1 | - |
| 4 | I4 | I3 | I2 | I1 |
| 5 | I5 | I4 | I3 | I2 |
| 6 | I6 | I5 | I4 | I3 |
| 7 | - | I6 | I5 | I4 |
| 8 | - | - | I6 | I5 |
| 9 | - | - | - | I6 |

```csharp
public class BranchPrediction1
{
    public static int[] RandomNumbers = Enumerable
        .Range(0, 1_000_000)
        .OrderBy(_ => Guid.NewGuid())
        .ToArray();

    public static int[] SortedNumbers = RandomNumbers.OrderBy(n => n).ToArray();

    [Benchmark]
    public int SumRandomIfElse()
    {
        int sum = 0;
        foreach (var number in RandomNumbers)
        {
            if (number < 500_000)
                sum += number;
            else
                sum -= number;
        }
        return sum;
    }

    [Benchmark]
    public int SumSortedIfElse()
    {
        int sum = 0;
        foreach (var number in SortedNumbers)
        {
            if (number < 500_000)
                sum += number;
            else
                sum -= number;
        }
        return sum;
    }
}
```

```csharp
public class BranchPrediction2
{
    public static int[] RandomNumbers = Enumerable
        .Range(0, 1_000_000)
        .OrderBy(_ => Guid.NewGuid())
        .ToArray();

    public static int[] SortedNumbers = RandomNumbers.OrderBy(n => n).ToArray();

    [Benchmark]
    public int SumRandomIfElse()
    {
        int sum = 0;
        foreach (var number in RandomNumbers)
        {
            if (number % 2 == 0)
                sum += number;
            else
                sum -= number;
        }
        return sum;
    }

    [Benchmark]
    public int SumSortedIfElse()
    {
        int sum = 0;
        foreach (var number in SortedNumbers)
        {
            if (number % 2 == 0)
                sum += number;
            else
                sum -= number;
        }
        return sum;
    }
}
```

In some cases code can be rewritten so that it produces the same result, but doesn't have branching at all.

Consider original code was only counting even numbers:

```csharp
public class BranchPrediction3
{
    public static int[] RandomNumbers = Enumerable
        .Range(0, 1_000_000)
        .OrderBy(_ => Guid.NewGuid())
        .ToArray();

    public static int[] SortedNumbers = RandomNumbers.OrderBy(n => n).ToArray();

    [Benchmark]
    public int CountEvenRandom()
    {
        int count = 0;

        foreach (var number in RandomNumbers)
        {
            if (number % 2 == 0)
                count++;
        }

        return count;
    }

    [Benchmark]
    public int CountEvenSorted()
    {
        int count = 0;

        foreach (var number in SortedNumbers)
        {
            if (number % 2 == 0)
                count++;
        }

        return count;
    }
}
```

In can be updated to produced same result:

```csharp
public class BranchPrediction4
{
    public static int[] RandomNumbers = Enumerable
        .Range(0, 1_000_000)
        .OrderBy(_ => Guid.NewGuid())
        .ToArray();

    public static int[] SortedNumbers = RandomNumbers.OrderBy(n => n).ToArray();

    [Benchmark]
    public int CountEvenRandom()
    {
        int count = 0;

        foreach (var number in RandomNumbers)
        {
            if (number % 2 == 0)
                count++;
        }

        return count;
    }

    [Benchmark]
    public int CountEvenSorted()
    {
        int count = 0;

        foreach (var number in SortedNumbers)
        {
            if (number % 2 == 0)
                count++;
        }

        return count;
    }

    [Benchmark]
    public int CountEvenNoBranchRandom()
    {
        int count = 0;

        foreach (var number in RandomNumbers)
        {
            count += (number & 1) ^ 1;
        }

        return count;
    }

    [Benchmark]
    public int CountEvenNoBranchSorted()
    {
        int count = 0;

        foreach (var number in SortedNumbers)
        {
            count += (number & 1) ^ 1;
        }

        return count;
    }
}
```


## SIMD

```csharp
public class Simd1
{
    private readonly int[] RandomNumbers = Enumerable
        .Range(0, 800_000)
        .Select(_ => Random.Shared.Next(0, 10000))
        .ToArray();

    [Benchmark]
    public long BaselineSum()
    {
        long sum = 0;
        foreach (var number in RandomNumbers)
        {
            sum += number;
        }
        return sum;
    }

    [Benchmark]
    public long Avx2Sum_Safe()
    {
        if (!Avx2.IsSupported)
            throw new PlatformNotSupportedException();

        Vector256<int> sumVector = Vector256<int>.Zero;

        for (var i = 0; i <= RandomNumbers.Length - 8; i += 8)
        {
            var vector = Vector256.Create(RandomNumbers, i);
            sumVector = Avx2.Add(sumVector, vector);
        }

        long sum = 0;
        for (int j = 0; j < 8; j++)
        {
            sum += sumVector.GetElement(j);
        }

        return sum;
    }

    [Benchmark]
    public long Avx2Sum_Unsafe()
    {
        if (!Avx2.IsSupported)
            throw new PlatformNotSupportedException();

        Vector256<int> sumVector = Vector256<int>.Zero;
        int i = 0;

        unsafe
        {
            fixed (int* ptr = RandomNumbers)
            {
                for (; i <= RandomNumbers.Length - 8; i += 8)
                {
                    var vector = Avx.LoadVector256(ptr + i);
                    sumVector = Avx2.Add(sumVector, vector);
                }
            }
        }

        long sum = 0;
        for (int j = 0; j < 8; j++)
        {
            sum += sumVector.GetElement(j);
        }

        return sum;
    }

    [Benchmark]
    public long Sse2Sum_Unsafe()
    {
        if (!Sse2.IsSupported)
            throw new PlatformNotSupportedException();

        Vector128<int> sumVector = Vector128<int>.Zero;
        int i = 0;

        unsafe
        {
            fixed (int* ptr = RandomNumbers)
            {
                for (; i <= RandomNumbers.Length - 4; i += 4)
                {
                    var vector = Sse2.LoadVector128(ptr + i);
                    sumVector = Sse2.Add(sumVector, vector);
                }
            }
        }

        long sum = 0;
        for (int j = 0; j < 4; j++)
        {
            sum += sumVector.GetElement(j);
        }

        return sum;
    }

    [Benchmark]
    public long Avx512Sum_Unsafe()
    {
        if (!Avx512F.IsSupported)
            throw new PlatformNotSupportedException();

        Vector512<int> sumVector = Vector512<int>.Zero;
        int i = 0;

        unsafe
        {
            fixed (int* ptr = RandomNumbers)
            {
                for (; i <= RandomNumbers.Length - 16; i += 16)
                {
                    var vector = Avx512F.LoadVector512(ptr + i);
                    sumVector = Avx512F.Add(sumVector, vector);
                }
            }
        }

        long sum = 0;
        for (int j = 0; j < 16; j++)
        {
            sum += sumVector.GetElement(j);
        }

        return sum;
    }
}
```